In [2]:
import os
import random
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np
import pandas as pd
from ultralytics import YOLO
import shutil

In [2]:
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, xmin

In [ ]:
def plot(image_paths, label_paths, num_samples=4):
    all_images = []
    all_images.extend(glob.glob(image_paths+'/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/*.JPG'))
    all_images.sort()
    num_images = len(all_images)
    plt.figure(figsize=(15,12))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image_name = all_images[j]
        image_name = '.'.join(image_name.split(os.path.sep)[-1].split('.')[:-1])
        image = cv2.imread(all_images[j])
        with open(os.path.join(label_paths, image_name+'.txt'), 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                bbox_string = label_line [2:]
                x_c, y_c, w, h = bbox_string.split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w ,h])
                labels.append(label)
            result_image = plot_box(image, bboxes, labels)
            plt.subplot(2, 2, i+1)
            plt.imshow(result_image[:,:,::-1])
            plt.axis('off')
        plt.subplots_adjust(wspace=1)
        plt.tight_layout()
        plt.show()

In [ ]:
%%writefile tumor_v8.yaml
path: '/Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection/new'
train: 'train/images'
val: 'valid/images'
test: 'test/images'

#class names
names:
    0: 'type1'
    1: 'type2'
    2: 'type3'

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8s.pt')

In [ ]:
# Assuming `model` is your trained YOLO model and `test_images` is a list of your test images

for image in test_images:
    # Run the model on the test image
    boxes, confidences, class_ids = model.predict(image)

    # Now `boxes` contains the bounding boxes predicted by the model,
    # `confidences` contains the confidence scores for each box, and
    # `class_ids` contains the class IDs for each box.

    # You can compare these predictions to the ground truth to evaluate the model.

# new code from here

In [7]:
#since using only label 1 we need to rewrite the label files
#by removing label 0 and 2 and the scans without any tumor


In [79]:
def clean_data(data_path):
    os.makedirs('newTrain',exist_ok=True)
    os.makedirs(os.path.join(os.getcwd(),'newTrain', 'images'), exist_ok=True)
    os.makedirs(os.path.join(os.getcwd(), 'newTrain', 'labels'), exist_ok=True)

    labels_path = os.path.join(os.getcwd(), 'train', 'labels')
    img_path = os.path.join(os.getcwd(), 'train', 'images')

    new_labels_path = os.path.join(os.getcwd(), 'newTrain', 'labels')
    new_images_path = os.path.join(os.getcwd(), 'newTrain', 'images')
    
    for file_name in os.listdir(labels_path):
        file_path  = os.path.join(labels_path, file_name)
        with open(file_path, 'r') as f:
            image_file_name = file_name.replace('.txt', '.jpg')
            image_file_path = os.path.join(img_path, image_file_name)
            lines = f.readlines()
            if not lines:
                shutil.copy(file_path, new_labels_path)
                shutil.copy(image_file_path, new_images_path)
            else: 
                lines_id = [line for line in lines if line.startswith('1')]
                if lines_id:
                    with open(os.path.join(new_labels_path, file_name), 'w') as f: 
                        lines_id = lines_id[0].split(' ')
                        lines_id[0] = '0 '
                        ' '.join(lines_id)
                        f.writelines(lines_id)
                        shutil.copy(image_file_path, new_images_path)
            
    

In [91]:
def clean_data2(data_path, dir1, dir2):
    os.makedirs('newTrain',exist_ok=True)
    os.makedirs(os.path.join(os.getcwd(),dir2, 'images'), exist_ok=True)
    os.makedirs(os.path.join(os.getcwd(), dir2, 'labels'), exist_ok=True)

    labels_path = os.path.join(os.getcwd(), dir1, 'labels')
    img_path = os.path.join(os.getcwd(), dir1, 'images')

    new_labels_path = os.path.join(os.getcwd(), dir2, 'labels')
    new_images_path = os.path.join(os.getcwd(), dir2, 'images')
    
    for file_name in os.listdir(labels_path):
        file_path  = os.path.join(labels_path, file_name)
        with open(file_path, 'r') as f:
            image_file_name = file_name.replace('.txt', '.jpg')
            image_file_path = os.path.join(img_path, image_file_name)
            lines = f.readlines()
            if not lines:
                shutil.copy(file_path, new_labels_path)
                shutil.copy(image_file_path, new_images_path)
            else: 
                lines_id = [line for line in lines if line.startswith('1')]
                if lines_id:
                    with open(os.path.join(new_labels_path, file_name), 'w') as f: 
                        lines_id = lines_id[0].split(' ')
                        lines_id[0] = '0 '
                        lines_id = ' '.join(lines_id)
                        lines_id = lines_id.strip()
                        f.writelines(lines_id)
                        shutil.copy(image_file_path, new_images_path)
            
    

In [92]:
my_path ='/Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection/new'
clean_data2(my_path, 'train', 'newTrain')
clean_data2(my_path, 'valid', 'newValid')
clean_data2(my_path, 'test', 'newTest')

In [80]:
clean_data('/Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection')

In [29]:
#now after the data is cleaned we can start training the model

In [98]:
%%writefile tumor_v8.yaml
path: '/Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection'
train: 'newTrain/images'
val: 'newValid/images'
test: 'newTest/images'
nc: 1

#class names
names:
    0: '1'

Overwriting tumor_v8.yaml


In [99]:
model = YOLO('yolov8m.pt')

In [100]:
EPOCHS = 8
BATCH = 256
IMG_SIZE = 160

In [101]:
results = model.train(
    data = 'tumor_v8.yaml',
    imgsz = IMG_SIZE,
    epochs = EPOCHS,
    batch = BATCH,
    name = 'output'
)

New https://pypi.org/project/ultralytics/8.2.3 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.11.0 torch-2.2.0 CPU (Apple M2 Pro)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=tumor_v8.yaml, epochs=8, time=None, patience=100, batch=256, imgsz=160, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=output17, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, 

train: Scanning /Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection/newTrain/labels... 6824 images, 79 backgrounds, 0 corrupt: 100%|██████████| 6824/6824 [00:01<00:00, 3634.25it/s]


train: New cache created: /Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection/newTrain/labels.cache


val: Scanning /Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection/newValid/labels... 1961 images, 17 backgrounds, 0 corrupt: 100%|██████████| 1961/1961 [00:00<00:00, 3246.93it/s]


val: New cache created: /Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection/newValid/labels.cache
Plotting labels to runs/detect/output17/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.002), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 160 train, 160 val
Using 0 dataloader workers
Logging results to runs/detect/output17
Starting training for 8 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        8/8         0G      1.322     0.8609      1.288        313        160: 100%|██████████| 27/27 [12:36<00:00, 28.03s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [01:33<00:00, 23.27s/it]

                   all       1961       1944      0.883      0.692      0.771      0.467



8 epochs completed in 1.904 hours.
Optimizer stripped from runs/detect/output17/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/output17/weights/best.pt, 52.0MB

Validating runs/detect/output17/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.11.0 torch-2.2.0 CPU (Apple M2 Pro)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [01:35<00:00, 23.95s/it]


                   all       1961       1944      0.883      0.692      0.771      0.467
Speed: 0.1ms preprocess, 47.9ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/output17
